In [3]:
import win32com.client
import pythoncom
import datetime, time# for sleep
import pandas as pd # command 로 시작할 때는 모듈 import 위치고려
from datetime import timedelta
from init_Stock import *

def __Init_Conn() :

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    class XASessionEventHandler:
        
        login_state = 0

        def OnLogin(self, code, msg):

            if code == "0000":
                print("로그인 성공")
                XASessionEventHandler.login_state = 1
            else:
                print("로그인 실패")

    instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)

    # 접속할 서버의 기본 주소는 'hts.ebestsec.co.kr'인데 모의 투자인 경우에는 'demo.ebestsec.co.kr'을 사용
    print("Server_Name : %s" % Server_Name)
    instXASession.ConnectServer(Server_Name, 20001)
#     instXASession.ConnectServer("hts.ebestsec.co.kr", 20001)
    instXASession.Login(id, passwd, cert_passwd, 0, 0)

    while XASessionEventHandler.login_state == 0:
        pythoncom.PumpWaitingMessages()

##################################################
# 증시대기자금 확인
##################################################

def __Check_Money_Flow() :

    # 예탁금 조회 t8428
    class XAQueryEventHandlerT8428:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT8428.query_state = 1

    instXAQueryT8428 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT8428)
    instXAQueryT8428.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t8428.res"

#     instXAQueryT8428.SetFieldData("t8428InBlock", "fdate", 0, 20200101) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
#     instXAQueryT8428.SetFieldData("t8428InBlock", "tdate", 0, 20200825)
    instXAQueryT8428.SetFieldData("t8428InBlock", "upcode", 0, "001") # 코스피 : 001, 코스닥 : 300
    instXAQueryT8428.SetFieldData("t8428InBlock", "cnt", 0, 100)

    instXAQueryT8428.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT8428.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

    count1 = instXAQueryT8428.GetBlockCount("t8428OutBlock1")

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
#     File_Name = "D:\\Python\\Log\\Money_Flow.txt"

    for i in range(count1):
        date = instXAQueryT8428.GetFieldData("t8428OutBlock1", "date", i)
        jisu = instXAQueryT8428.GetFieldData("t8428OutBlock1", "jisu", i) # 지수
        diff = instXAQueryT8428.GetFieldData("t8428OutBlock1", "diff", i) # 등락율
        volume = instXAQueryT8428.GetFieldData("t8428OutBlock1", "volume", i) # 거래량
        custmoney = instXAQueryT8428.GetFieldData("t8428OutBlock1", "custmoney", i) # 고객예탁금
        outmoney = instXAQueryT8428.GetFieldData("t8428OutBlock1", "outmoney", i) # 미수금
        trjango = instXAQueryT8428.GetFieldData("t8428OutBlock1", "trjango", i) # 신용잔고
        stkmoney = instXAQueryT8428.GetFieldData("t8428OutBlock1", "stkmoney", i) # 주식형 볼륨
#         print("날짜 : %s, 지수 : %s, 등락율 : %s, 거래량 : %s, 예탁금 : %s, 미수금 : %s, 신용잔고 : %s, 주식볼륨 : %s\n" % (date, jisu, diff, volume, custmoney, outmoney, trjango, stkmoney))
        print("%s, %s, %s, %s, %s, %s, %s, %s\n" % (date, jisu, diff, volume, custmoney, outmoney, trjango, stkmoney))
#         output.write("날짜 : %s, 지수 : %s, 등락율 : %s, 거래량 : %s, 예탁금 : %s, 미수금 : %s, 신용잔고 : %s, 주식볼륨 : %s\n" % (date, jisu, diff, volume, custmoney, outmoney, trjango, stkmoney))
#         output.write("%s, %s, %s, %s, %s, %s, %s, %s\n" % (date, jisu, diff, volume, custmoney, outmoney, trjango, stkmoney))
#     output.close()

#     return File_Name

if __name__ == '__main__' :

        print("# Init Connection")
        # output = open(File_Name, "a")
        __Init_Conn()
        
        __Check_Money_Flow()


# Init Connection
Server_Name : hts.ebestsec.co.kr
로그인 성공
20210511, 3209.43, -1.23, 1096717, 691600, 2599, 229222, 822065

20210510, 3249.30, 1.63, 1011267, 677666, 2774, 227818, 825778

20210507, 3197.20, 0.58, 989680, 679612, 2976, 226114, 818296

20210506, 3178.74, 1.00, 1239403, 720585, 3810, 228184, 814575

20210504, 3147.37, 0.64, 1283829, 739238, 3722, 230547, 811453

20210503, 3127.20, -0.66, 1041199, 779018, 3590, 232071, 805573

20210430, 3147.86, -0.83, 1359455, 584167, 3927, 233748, 808797

20210429, 3174.07, -0.23, 1222430, 594842, 3270, 235454, 811250

20210428, 3181.47, -1.06, 1412620, 735959, 2774, 234510, 811923

20210427, 3215.42, -0.07, 1534230, 699922, 2820, 233749, 817809

20210426, 3217.53, 0.99, 1298330, 686021, 3182, 233759, 816904

20210423, 3186.10, 0.27, 2253793, 676877, 3180, 233769, 812878

20210422, 3177.52, 0.18, 1752691, 694172, 2648, 231994, 814561

20210421, 3171.66, -1.52, 1767090, 671251, 2826, 230905, 810677

20210420, 3220.70, 0.68, 1296160, 757884